# ATLAS Explorer: Single-Core Experiment (Customer Notebook)

Welcome! This guided notebook walks you through running a *single-core* ATLAS Explorer experiment without needing to know Python programming.

You will:
1. Verify prerequisites
2. Configure (or auto-detect) your credentials
3. Set a few experiment parameters (file, core type, etc.)
4. Run the experiment
5. View key results (Total Cycles + summary table)
6. (Optional) Explore derived metrics

Tips:
- Edit only the clearly marked parameter cells (grey code boxes with simple assignments)
- Re‑run a cell with Shift+Enter
- If something fails, read the printed message and adjust the parameters

---


In [1]:
"""Preamble: core imports used later. No need to modify."""
import os, json, locale, datetime
from pathlib import Path
import pandas as pd  # used for tabular display
from dotenv import load_dotenv

# Load any .env for convenience (optional)
load_dotenv()
locale.setlocale(locale.LC_ALL, "")

print("Environment initialized. Proceed to the next cell.")

Environment initialized. Proceed to the next cell.


## 1. Check Project & Existing Experiments
This cell will:
- Ensure the working directory is the repository root
- List any prior experiment runs in `myexperiments/`

You don't need to edit the code—just run it.

In [2]:
# (No edits needed) Detect repo root and list existing experiment runs.
from pathlib import Path
import os, datetime

cwd = Path.cwd()
repo_root = None
for p in [cwd, *cwd.parents]:
    if (p / "pyproject.toml").exists() or (p / ".git").exists():
        repo_root = p
        break
repo_root = repo_root or cwd
os.chdir(repo_root)
print(f"Working directory set to repo root: {repo_root}")

exp_root = repo_root / "myexperiments"
rows = []
if exp_root.exists():
    for run_dir in sorted(exp_root.iterdir()):
        if run_dir.is_dir():
            summary = next(run_dir.rglob("reports/summary/summary.json"), None)
            mtime = datetime.datetime.fromtimestamp(run_dir.stat().st_mtime)
            rows.append({
                "name": run_dir.name,
                "summary": str(summary) if summary else "-",
                "modified": mtime.isoformat(timespec="seconds"),
            })
    if not rows:
        print("No experiments found yet under 'myexperiments'.")
else:
    print("No 'myexperiments' directory found yet.")

if rows:
    df = pd.DataFrame(rows).sort_values("modified", ascending=False)
    display(df)

Working directory set to repo root: /Users/jschroeder/Documents/code_repos/mips-ae-pylib


,name,summary,modified
1,I8500_(2_threads)_250808_103012,/Users/jschroeder/Documents/code_repos/mips-ae...,2025-08-08T10:34:36
0,I8500_(1_thread)_250808_102301,/Users/jschroeder/Documents/code_repos/mips-ae...,2025-08-08T10:27:58


## Project root & experiments
This notebook will first switch the working directory to the repository root and list any existing experiments under `myexperiments/`.

## 2. Configure Credentials
ATLAS Explorer needs your API key, channel, and region. The notebook will auto-detect them from:
1. Environment variable `MIPS_ATLAS_CONFIG` (format: apikey:channel:region)
2. Config file `~/.config/mips/atlaspy/config.json`

If neither is found you'll enter them below.

In [3]:
# Enter ONLY if prompted. Run once.
from pathlib import Path
import os, json
from dotenv import load_dotenv

load_dotenv()
CONFIG_ENV = "MIPS_ATLAS_CONFIG"
cfg_file = Path.home() / ".config/mips/atlaspy/config.json"

if os.environ.get(CONFIG_ENV):
    print("Credentials detected from environment variable MIPS_ATLAS_CONFIG.")
elif cfg_file.exists():
    try:
        data = json.loads(cfg_file.read_text())
        os.environ[CONFIG_ENV] = f"{data['apikey']}:{data['channel']}:{data['region']}"
        print(f"Credentials loaded from {cfg_file}")
    except Exception as e:
        print(f"Could not parse {cfg_file}: {e}")

if not os.environ.get(CONFIG_ENV):
    print("No saved credentials found. Please fill the three fields below and re-run this cell.")
    ae_apikey   = ""  # REQUIRED e.g. "your-api-key"
    ae_channel  = "development"  # or your channel
    ae_region   = ""  # REQUIRED e.g. "us-west-2"
    persist_to_file = True

    if not ae_apikey or not ae_channel or not ae_region:
        print("Waiting for input... (fill ae_apikey, ae_channel, ae_region above)")
    else:
        os.environ[CONFIG_ENV] = f"{ae_apikey}:{ae_channel}:{ae_region}"
        print("Session credentials set.")
        if persist_to_file:
            cfg_file.parent.mkdir(parents=True, exist_ok=True)
            cfg_file.write_text(json.dumps({"apikey": ae_apikey, "channel": ae_channel, "region": ae_region}, indent=2))
            print(f"Saved to {cfg_file}")
else:
    print("Ready.")

Credentials detected from environment variable MIPS_ATLAS_CONFIG.
Ready.


## 3. Experiment Overview
We'll run a single workload (ELF) on a specified core configuration. Adjust parameters next, then run the experiment.

### Prerequisites
- Credentials configured (step 2)
- Example ELF present under `resources/` (default provided)
- No changes needed below unless you want a different ELF or core

In [4]:
# 3.a EDITABLE PARAMETERS
elf = "resources/mandelbrot_rv64_O0.elf"   # Path to workload ELF
expdir = "myexperiments"                   # Folder to store runs
core = "I8500_(1_thread)"                  # Core type (matches script example)
channel = "development"                    # Channel (can leave as-is)
apikey = None                               # Usually leave None (auto from config)
region = None                               # Usually leave None (auto from config)
verbose = False                             # Set True for detailed logs

# Optional export format: None | 'json' | 'markdown' | 'html' | 'rich-html' | 'zip'
export = None
out = None  # Optional target path for exported artifact
print("Parameters set. Modify above as needed.")

Parameters set. Modify above as needed.


In [5]:
# 3.b Environment validation (no edits needed)
import os, locale
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
locale.setlocale(locale.LC_ALL, "")

config_env = os.environ.get("MIPS_ATLAS_CONFIG")
config_file = Path.home() / ".config/mips/atlaspy/config.json"
if not apikey and not config_env and not config_file.exists():
    print("NOTE: Run 'uv run atlasexplorer/atlasexplorer.py configure' or set MIPS_ATLAS_CONFIG before running.")
else:
    print("Credentials appear available. Proceed.")

Credentials appear available. Proceed.


In [6]:
# 4. Run the experiment (this uploads, executes, downloads results)
from atlasexplorer.atlasexplorer import AtlasExplorer, Experiment

print("Starting experiment run...")
aeinst = AtlasExplorer(apikey, channel, region, verbose=verbose)
experiment = Experiment(expdir, aeinst, verbose=verbose)
experiment.addWorkload(os.path.abspath(elf))
experiment.setCore(core)
experiment.run()

# Total cycles (matches example script output)
try:
    total_cycles = experiment.getSummary().getTotalCycles()
    print(f"Total Cycles: {total_cycles}")
except Exception:
    print("Could not retrieve total cycles from summary object.")

# Locate latest summary.json
from pathlib import Path
exp_path = Path(expdir)
summary_candidates = list(exp_path.rglob("summary/summary.json"))
if not summary_candidates:
    raise FileNotFoundError("No summary.json found under experiment directory")
summary_candidates.sort(key=lambda p: p.stat().st_mtime, reverse=True)
summary_path = summary_candidates[0]
print(f"Latest summary: {summary_path}")

Starting experiment run...
Available versions: latest, ST-2025-07-16-171806
No report directory found:summary
Total Cycles: 253629
Latest summary: myexperiments/I8500_(1_thread)_250811_105319/I8500_(1_thread)_250811_105319/reports/summary/summary.json


In [7]:
# 5. View summary (tabular)
import json, pandas as pd
with open(summary_path) as f:
    summary_data = json.load(f)

if isinstance(summary_data, dict):
    df = pd.DataFrame([summary_data])
elif isinstance(summary_data, list):
    df = pd.DataFrame(summary_data)
else:
    df = None

if df is not None:
    display(df.head())
else:
    print(summary_data)

print("Displayed summary (first rows if large).")

,Statistics,vis,siminfo,report_metadata
0,{'Summary Performance Report': {'Total Cycles ...,"{'hidden': 0, 'support': 1000000, 'detail': 10...","{'name': 'Shinro RISC-V Perf Model ', 'sim_ver...","{'arch': 'I8500_(1_thread)', 'report_format': ..."


Displayed summary (first rows if large).


In [ ]:
# 6.a Load metrics from summary.json (edit summary_path if you want another file)
import json, locale, math
from pathlib import Path
import pandas as pd

# If you have an external summary file, override here, e.g.:
# summary_path = Path("/full/path/to/other/summary.json")

if not 'summary_path' in globals():
    raise RuntimeError("summary_path not found (run experiment first or set manually)")

with open(summary_path) as f:
    raw = json.load(f)

# Expected structure: raw['Statistics']['Summary Performance Report']
try:
    metrics_root = raw['Statistics']['Summary Performance Report']
except KeyError:
    raise KeyError("summary.json does not have expected Statistics -> Summary Performance Report structure")

rows = []
for key, entry in metrics_root.items():
    if key == 'ordered_keys':
        continue
    val = entry.get('val')
    unit = entry.get('unit') or entry.get('units') or ''
    rows.append({
        'Metric': key,
        'Value': val,
        'Unit': unit,
    })

metrics_df = pd.DataFrame(rows)

# Nicely format large integers with grouping
locale.setlocale(locale.LC_ALL, "")
def fmt(v):
    if isinstance(v, int):
        try:
            return locale.format_string('%d', v, grouping=True)
        except Exception:
            return v
    if isinstance(v, float):
        if math.isfinite(v):
            return f"{v:,.4g}"
    return v

metrics_df['Formatted'] = metrics_df['Value'].map(fmt)
metrics_df.head(len(metrics_df))

In [ ]:
# 6.b OPTIONAL: Simple text filter (set filter_text then run)
filter_text = ''  # e.g. 'Cycles' or 'Instructions'
filtered = metrics_df
if filter_text:
    ft = filter_text.lower()
    filtered = metrics_df[metrics_df['Metric'].str.lower().str.contains(ft)]
filtered.head(len(filtered))

### 6. Explore Metrics (Interactive)
Below you can load metrics from the `summary.json` and interactively filter or sort them.
If you have a different `summary.json` path, set it in the next code cell.